NameError: name 'model' is not defined

Lets start by building the TSP solver using the Google OR tools interfaces. We start by defining a class that takes the dictionary of nodes and edges as inputs. 

We then make a function the creates the model data by passing it to the OR tools interface. We pass in the location identifiers as a tuple $(s_n,d_n)$

UsageError: Cell magic `%%add_to` not found.


## Solving TSP using google OR pickup and delivery system 

While creating the model and transformation to solve as a TSP for a single vehicle we discovered that google-OR tools actually has an efficient library for solving problems where edges must be enforced. They describe this problem as the pickup/delivery TSP problem https://developers.google.com/optimization/routing/pickup_delivery. 


In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

class TSPORSolver:

    def __init__(self,nodes,edges,deliveryEdges,VN):
        self.nodes = nodes 
        self.edges = edges
        self.deliveryEdges = deliveryEdges
        self.VN = VN

    def create_data_model(self):
        data = {}
        data['locations'] = nodes
        data['pickups_deliveries'] = self.deliveryEdges 
        data['num_vehicles'] = self.VN
        data['depot'] = 0
        return data

    def create_distance_callback(self,data, manager):
        distances_ = {}
        index_manager_ = manager
        for from_counter, from_node in enumerate(data['locations']):
            distances_[from_counter] = {}
            for to_counter, to_node in enumerate(data['locations']):
                if from_counter == to_counter:
                    distances_[from_counter][to_counter] = 0
                else:
                    distances_[from_counter][to_counter] = self.edges.loc[from_node,to_node]        
        def distance_callback(from_index, to_index):
            from_node = index_manager_.IndexToNode(from_index)
            to_node = index_manager_.IndexToNode(to_index)
            return distances_[from_node][to_node]
        return distance_callback

    """def print_solution(self,manager, routing, solution):
        print('Objective: {}'.format(solution.ObjectiveValue()))
        index = routing.Start(0)
        plan_output = 'Route:\n'
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} ->'.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        plan_output += ' {}\n'.format(manager.IndexToNode(index))
        plan_output += 'Objective: {}m\n'.format(route_distance)
        print(plan_output)
        """
    def print_solution(self,data, manager, routing, solution):
        """Prints solution on console."""
        print(f'Objective: {solution.ObjectiveValue()}')
        max_route_distance = 0
        for vehicle_id in range(data['num_vehicles']):
            index = routing.Start(vehicle_id)
            plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
            route_distance = 0
            while not routing.IsEnd(index):
                plan_output += ' {} -> '.format(manager.IndexToNode(index))
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(
                    previous_index, index, vehicle_id)
            plan_output += '{}\n'.format(manager.IndexToNode(index))
            plan_output += 'Distance of the route: {}m\n'.format(route_distance)
            print(plan_output)
            max_route_distance = max(route_distance, max_route_distance)
        print('Maximum of the route distances: {}m'.format(max_route_distance))

    def solve(self):
        data = self.create_data_model()
        manager = pywrapcp.RoutingIndexManager(len(data['locations']),
                                            data['num_vehicles'], data['depot'])
        routing = pywrapcp.RoutingModel(manager)

        distance_callback = self.create_distance_callback(data, manager)
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            200000,  # vehicle maximum travel distance
            True,  # start cumul to zero
            dimension_name)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)

        """
        for request in data['pickups_deliveries']:
            print(request[0])
            pickup_index = manager.NodeToIndex(request[0])
            delivery_index = manager.NodeToIndex(request[1])
            routing.AddPickupAndDelivery(pickup_index, delivery_index)
            routing.solver().Add(
                routing.VehicleVar(pickup_index) == routing.VehicleVar(
                    delivery_index))
            routing.solver().Add(
                distance_dimension.CumulVar(pickup_index) <=
                distance_dimension.CumulVar(delivery_index))
        """
       

        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
        search_parameters.time_limit.seconds= 10


        assignment = routing.SolveWithParameters(search_parameters)

        if assignment:
           return self.print_solution(data,manager, routing, assignment)

In [ ]:
nodes = pd.concat([demand,supply])['id'].to_list()
newEdges = []
newEdgeWeights = []
startGeo = []
endGeo = []

In [ ]:
deliveryEdges = []
for v in model.X:
    if model.X[v].value !=0 and v[0] != "-1" and v[1] != "-2":
        deliveryEdges.append((v[0],v[1]))
deliveryEdges[1]

In [ ]:

len(deliveryEdges)
d = {}
for x in deliveryEdges:
    d.update({x[0]: d.get(x[0],0) + 1 })
distanceArray = []
distances = distances.astype(int)
for date, new_df in distances.groupby(level=0):
    distanceArray.append(new_df.to_list())
distanceArray

In [ ]:

#Reverse Edges 
for index , row in demand.iterrows():
    for i , r in supply.iterrows():
        if row['id'] != r['id']:
            newEdges.append((row['id'],r['id']))
            startGeo.append(row.geometry)
            endGeo.append(r.geometry)


In [ ]:
for index,row in supply.iterrows():
    for i,r in supply.iterrows():
        newEdges.append((row['id'],r['id']))
        startGeo.append(row.geometry)
        endGeo.append(r.geometry)

In [ ]:
for index,row in demand.iterrows():
    for i,r in demand.iterrows():
        newEdges.append((row['id'],r['id']))
        startGeo.append(row.geometry)
        endGeo.append(r.geometry)

In [ ]:
index = pd.MultiIndex.from_tuples(newEdges,names =['edgeStart','edgeEnd'])

d = {'startPoint': startGeo,'endPoint':endGeo}
newEdgesFrame = pd.DataFrame(data=d,index = index)
newEdgesFrame.head()

In [ ]:
x =  gpd.GeoSeries(newEdgesFrame['startPoint'],crs=pyproj.CRS("EPSG:27700"))
y = gpd.GeoSeries(newEdgesFrame['endPoint'],crs= pyproj.CRS("EPSG:27700"))
distances = x.distance(y,align=False)


newEdgesFrame['weight'] = distances
newEdgesFrame =  newEdgesFrame.drop("startPoint", axis=1)
newEdgesFrame = newEdgesFrame.drop("endPoint", axis=1)
newEdgesFrame

In [ ]:
VRPEdges = stationEdges['weight']
VRPEdges = pd.concat([VRPEdges,newEdgesFrame['weight']])
VRPEdges= VRPEdges.astype(int)
VRPEdges

In [ ]:
VRPEdges.loc['86', '-2']